In [1]:
# import os

# for speaker_id in os.listdir('ClonedAudios'):
#     path = f"ClonedAudios/{speaker_id}"
#     if os.path.isdir(path) and 'ipynb' not in path:
#         n_files = len(os.listdir(path))
#         print(f"{path} files: {n_files}")
#         print('------------------------')

In [2]:
import os
import pandas as pd

total_cloned = 0 
for speaker_id in os.listdir('ClonedAudiosout'):
    path = f"ClonedAudiosout/{speaker_id}"
    if os.path.isdir(path) and 'ipynb' not in path:
        for folder in os.listdir(path):
            complete_path = f"{path}/{folder}"
            files = os.listdir(complete_path)
            n_files = len(files)
            total_cloned += n_files
            print(f"{complete_path} files: {n_files}")
            print('------------------------')
            
    if 'csv' in path:
        print(f"CSV rows: {len(pd.read_csv(path))}")
        print('-----------')
        

total_audios = 0
speaker_ids = [speaker_id for speaker_id in os.listdir('ClonedAudios') if 'ipynb' not in speaker_id]

for speaker_id in speaker_ids:
    other_speaker_ids = [other_speaker_id for other_speaker_id in speaker_ids if other_speaker_id != speaker_id]
    
    for other_speaker_id in other_speaker_ids:
        n_files = min(len(os.listdir(f"ClonedAudios/{other_speaker_id}")), 100)
        total_audios += n_files
    
print(f'Audios cloned at moment in total: {total_cloned}')
print(f'Total audios clonable: {total_audios}')
print(f'{100*total_cloned / total_audios} % of the total')

ClonedAudiosout/3922/locutor4 files: 100
------------------------
ClonedAudiosout/3922/locutor5 files: 100
------------------------
ClonedAudiosout/3922/locutor1 files: 100
------------------------
ClonedAudiosout/3922/3638 files: 100
------------------------
ClonedAudiosout/3922/locutor2 files: 100
------------------------
ClonedAudiosout/3922/8088 files: 100
------------------------
ClonedAudiosout/3922/locutor6 files: 100
------------------------
ClonedAudiosout/3922/3307 files: 100
------------------------
ClonedAudiosout/3922/8699 files: 101
------------------------
ClonedAudiosout/3922/locutor3 files: 100
------------------------
ClonedAudiosout/3922/6209 files: 100
------------------------
ClonedAudiosout/locutor4/3922 files: 100
------------------------
ClonedAudiosout/locutor4/locutor5 files: 100
------------------------
ClonedAudiosout/locutor4/locutor1 files: 100
------------------------
ClonedAudiosout/locutor4/3638 files: 100
------------------------
ClonedAudiosout/locuto

In [3]:
import wave
def get_audio_length(file_path):
    with wave.open(file_path, 'rb') as audio_file:
        # Get the number of frames and the frame rate
        num_frames = audio_file.getnframes()
        frame_rate = audio_file.getframerate()

        # Calculate the duration in seconds
        duration = num_frames / frame_rate

    return duration

In [4]:
# 1. Controlar que si el audio ya está clonado no se vuelva a clonar.
# 2. Guardar el metadata csv en cada audio clonado
# 3. Leer el metadata csv antes de ejecutar (ese try except hacerlo bien)

import os
import shutil
import time
import random
import pandas as pd
from resemblyzer import VoiceEncoder, preprocess_wav
from pathlib import Path

encoder = VoiceEncoder()
speaker_ids = [speaker_id for speaker_id in os.listdir('ClonedAudios') if not speaker_id.startswith('.')]

try:
    metadata_so_vits = pd.read_csv('ClonedAudiosout/metadata_so_vits.csv')
    print('CSV loaded')
except:
    metadata_so_vits = pd.DataFrame()
    print('Dataframe created')

for speaker_id in speaker_ids:
    logs_dir = f"logs/{speaker_id}/"
    config = f"configs/configs{speaker_id}/44k/config.json"
    
    index = speaker_ids.index(speaker_id)
    other_speaker_ids = speaker_ids[:index] + speaker_ids[index+1:]

    reference_audios_path = f"ClonedAudios/{speaker_id}"
    reference_audios_list = os.listdir(reference_audios_path)
    
    for other_speaker_id in other_speaker_ids:
        wav_files = os.listdir(f"ClonedAudios/{other_speaker_id}")
        ################################################################
        wav_files = wav_files[:100]
        ################################################################
        names_aux = [wav_file[:-4] for wav_file in wav_files]
        
        os.makedirs(f"ClonedAudiosout/{speaker_id}/{other_speaker_id}", exist_ok=True)

        metadata_path = f"OriginalAudios/metadata_{other_speaker_id}.txt"
        metadata_other_speaker = pd.read_csv(metadata_path, sep='|', header=None, names=['Audio ID', 'Text'])

        for name_aux in names_aux:
            NAME = f"ClonedAudios/{other_speaker_id}/{name_aux}"
            file_path = f"ClonedAudiosout/{speaker_id}/{other_speaker_id}/{name_aux}.out.wav"
            
            if not os.path.exists(file_path):
                t1 = time.time()
                !svc infer {NAME}.wav -m {logs_dir} -c {config}
                t2 = time.time()
    
                file_path = f"ClonedAudiosout/{speaker_id}/{other_speaker_id}/{name_aux}.out.wav"
                shutil.move(f"{NAME}.out.wav", file_path)
                
                RTF = (t2 - t1) / (get_audio_length(file_path))
                
                reference_audio = random.choice(reference_audios_list)
                reference_audio_path = os.path.join(reference_audios_path, reference_audio)
                
                reference_audio_fpath = Path(reference_audio_path)
                reference_audio_wav = preprocess_wav(reference_audio_fpath)
                reference_audio_embed = encoder.embed_utterance(reference_audio_wav)
            
                synthesized_audio_fpath = Path(file_path)
                synthesized_audio_wav = preprocess_wav(synthesized_audio_fpath)
                synthesized_audio_embed = encoder.embed_utterance(synthesized_audio_wav)
            
                SECS = reference_audio_embed @ synthesized_audio_embed
                
                sentence = metadata_other_speaker[metadata_other_speaker['Audio ID'] == name_aux]
    
                if sentence.empty:
                    text = ''
                else:
                    text = sentence.reset_index().iloc[0]['Text']
                
                new_row = pd.DataFrame({'Reference audio path': [reference_audio_path],
                                        'Synthesized audio path': [file_path],
                                        'Text': [text],
                                        'Speaker ID': [speaker_id],
                                        'Original speaker ID': [other_speaker_id],
                                        'RTF': [RTF],
                                        'SECS': [SECS]
                                        })
            
                metadata_so_vits = pd.concat([metadata_so_vits, new_row])
                metadata_so_vits.reset_index(drop=True, inplace=True)
                metadata_so_vits.to_csv('ClonedAudiosout/metadata_so_vits.csv', index=False)

Loaded the voice encoder model on cuda in 1.95 seconds.
CSV loaded


In [5]:
met = pd.read_csv('ClonedAudiosout/metadata_so_vits.csv')
met

,Reference audio path,Synthesized audio path,Text,Speaker ID,Original speaker ID,RTF,SECS
0,ClonedAudios/3922/3922_715_000006_000003.wav,ClonedAudiosout/3922/locutor4/audio_22492_2249...,—Le prepararemos un baño y le proporcionaremo...,3922,locutor4,1.242560,0.612966
1,ClonedAudios/3922/3922_715_000023_000005.wav,ClonedAudiosout/3922/locutor4/audio_16440.out.wav,Siempre han existido personas que han gustado...,3922,locutor4,1.019879,0.735934
2,ClonedAudios/3922/3922_715_000118_000003.wav,ClonedAudiosout/3922/locutor4/audio_5732.out.wav,Pero es evidente que los viejos relatos bíbli...,3922,locutor4,0.665137,0.676058
3,ClonedAudios/3922/3922_715_000071_000001.wav,ClonedAudiosout/3922/locutor4/audio_17514_1751...,"Así, se dijo en mil novecientos treinta y cua...",3922,locutor4,0.760370,0.675240
4,ClonedAudios/3922/3922_723_000015_000000.wav,ClonedAudiosout/3922/locutor4/audio_24488_2448...,Clotilde dio un manotazo a una pila de libros...,3922,locutor4,0.908556,0.643677
...,...,...,...,...,...,...,...
13205,ClonedAudios/6209/6209_34601_000068_000032.wav,ClonedAudiosout/6209/locutor3/audio_1512.out.wav,"La verdad es que Pedro se preocupaba por mí, a...",6209,locutor3,0.979441,0.838969
13206,ClonedAudios/6209/6209_34601_000091_000003.wav,ClonedAudiosout/6209/locutor3/audio_154.out.wav,"Da risa, pero todavía me acuerdo de todo eso y...",6209,locutor3,0.739581,0.814811
13207,ClonedAudios/6209/6209_34601_000163_000012.wav,ClonedAudiosout/6209/locutor3/audio_3020.out.wav,"—Farfullaba, una y otra vez: «No quiero morir ...",6209,locutor3,0.741628,0.729366
13208,ClonedAudios/6209/6209_34601_000096_000060.wav,ClonedAudiosout/6209/locutor3/audio_1263.out.wav,"Cuando coincidía, hablaba sin parar, yo creo q...",6209,locutor3,1.011268,0.829923


In [6]:
from IPython.display import Audio, display

ref = met.loc[2, 'Reference audio path']
syn = met.loc[2, 'Synthesized audio path']
display(Audio(ref))
display(Audio(syn))

In [7]:
# Remove repeated audios from the csv

# print(f"Total audios cloned: {total_cloned}")
# print(f"Len metadata csv with repeated files: {len(met)}")
# synthesized_audios_csv = met['Synthesized audio path']
# strange_files = [file for file in synthesized_audios_csv if 'out.out' in file]

# for index, row in met.iterrows():
#     synthesized_audio_path = row['Synthesized audio path']
    
#     if synthesized_audio_path in strange_files:
#        met = met.drop(index)

# print(f"Len metadata csv without repeated files: {len(met)}")

# met.to_csv('ClonedAudiosout/metadata_so_vits.csv', index=False)



In [1]:
# Zip folders
import shutil
folder_path = 'OriginalAudios'
shutil.make_archive(folder_path, 'zip', folder_path)

folder_path = 'ClonedAudiosout'
shutil.make_archive(folder_path, 'zip', folder_path)

'/home/jorge/VoiceCloning/ClonedAudiosout.zip'